### Forecast Visualization

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display

### Evaluation metrics summary

In [10]:
RESULTS_DIR = '../results/forecasts/'

# Load all model directories
model_dirs = [d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, d))]

all_metrics = []

# Load and combine metrics
for model_name in model_dirs:
    metrics_path = os.path.join(RESULTS_DIR, model_name, 'metrics_summary.csv')
    if os.path.exists(metrics_path):
        df = pd.read_csv(metrics_path)
        df['model'] = model_name
        all_metrics.append(df)

metrics_df = pd.concat(all_metrics, ignore_index=True)


In [11]:
# Summary table: average metrics per model
summary_df = metrics_df.groupby('model').agg({'mape': 'mean', 'rmse': 'mean'}).reset_index()
summary_df = summary_df.sort_values(by='mape')

print("\n📊 Model Comparison Table (Averaged over all products):")
display(summary_df)


📊 Model Comparison Table (Averaged over all products):


,model,mape,rmse
1,NBEATSModel,45.189948,0.185066
0,ARIMA,47.705418,0.202689
2,Prophet,48.281565,0.200658
4,RandomForest,48.907590,0.187342
5,TCNModel,48.979249,0.187064
3,RNNModel,53.221070,0.224345


### Interactive Forecast Visualisation

In [13]:
# === Interactive Forecast Plot ===
def plot_forecast_vs_truth(model_name, product_id):
    forecast_path = os.path.join(RESULTS_DIR, model_name, f'forecast_product_{product_id}.csv')
    truth_path = os.path.join(RESULTS_DIR, model_name, f'ground_truth_product_{product_id}.csv')

    if not os.path.exists(forecast_path) or not os.path.exists(truth_path):
        print(f"❌ No forecast or ground truth data for model {model_name} product {product_id}")
        return

    forecast_df = pd.read_csv(forecast_path, index_col=0, parse_dates=True)
    truth_df = pd.read_csv(truth_path, index_col=0, parse_dates=True)

    plt.figure(figsize=(10, 5))
    plt.plot(truth_df, label='Ground Truth', color='blue')
    plt.plot(forecast_df, label='Forecast', color='orange')
    plt.title(f'{model_name} - Product {product_id} Forecast vs. Ground Truth')
    plt.xlabel('Date')
    plt.ylabel('Units Sold')
    plt.legend()
    plt.grid(True)
    plt.show()

In [14]:
model_selector = widgets.Dropdown(options=model_dirs, description='Model:')
product_selector = widgets.IntSlider(value=0, min=0, max=10, step=1, description='Product ID:')  # adjust max as needed

interactive_plot = widgets.interact(plot_forecast_vs_truth, model_name=model_selector, product_id=product_selector)

interactive(children=(Dropdown(description='Model:', options=('ARIMA', 'TCNModel', 'RandomForest', 'Prophet', …